# GloVe

```{note}
上下文窗口内的词共现可以携带丰富的语义信息。<br/>
Skip-Gram模型的损失函数可以被解释为带权重的交叉熵损失函数，直接训练的话速度很慢。<br/>
Glove（Global Vectors for Word Representation）可以解释为带权重的平方损失。
```

## Skip-Gram的损失函数

假设在语料库中，词 $w_{i}$ 上下文窗口中词 $w_{j}$ 出现的次数为 $x_{ij}$，Skip-Gram模型预测的概率为 $q_{ij}$，那么损失函数可以写成：

$$-\sum_{i\in\mathcal{V}}\sum_{j\in\mathcal{V}}x_{ij}\log{q_{ij}}$$

另一方面，用 $x_{i}$ 表示词 $w_{i}$ 上下文单词的总数，$p_{ij}$ 为统计的条件概率 $x_{ij} / x_{i}$，损失函数可以重写为：

$$-\sum_{i\in\mathcal{V}}x_{i}\sum_{j\in\mathcal{V}}p_{ij}\log{q_{ij}}$$

$-\sum_{j\in\mathcal{V}}p_{ij}\log{q_{ij}}$ 是计算全局语料统计的条件分布 $p_{ij}$ 和模型预测的条件分布 $q_{ij}$ 的交叉熵。这一损失按 $x_{i}$ 加权。

此损失有两个缺点：

1. 正如我们之前所说的，计算 $q_{ij}$ 需要对整个词表进行求和，计算上非常昂贵。
2. 来自大型语料库的大量罕见事件往往被交叉熵损失建模，从而赋予过多的权重。

## GloVe

GloVe模型基于平方损失对Skip-Gram模型做了三个修改：

1. 使用变量$p_{ij}'=\log{x_{ij}}$和$q_{ij}'=\mathbf{u}_{j}^{T}\mathbf{v}_{i}$ 而非概率分布，使用平方损失而非交叉熵损失：$(\mathbf{u}_{j}^{T}\mathbf{v}_{i} - \log{x_{ij}})^{2}$
2. 为每个词 $w_{i}$ 添加两个标量模型参数：中心词偏置 $b_{i}$ 和上下文偏置 $c_{i}$
3. 使用在[0,1]间递增的权重函数 $h({x_{ij}})$

$$\sum_{i\in\mathcal{V}}\sum_{j\in\mathcal{V}}h({x_{ij}})(\mathbf{u}_{j}^{T}\mathbf{v}_{i} + b_{i} + c_{j} - \log{x_{ij}})^{2}$$

对于权重函数，建议的选择是：当 $x<c$（例如$c=100$）时，$h(x) = (x/c)^{\alpha}$（例如$\alpha=0.75$）；否则 $h(x)=1$。

在每次迭代中，可以随机抽样一小批量非零的 $x_{ij}$ 来计算梯度并更新模型参数，不需要遍历所有单词进行计算。

训练完成后，单词 $w_{i}$ 的词向量为 $\mathbf{u}_{i}$ 和 $\mathbf{v}_{i}$ 的平均。